# Some experiments regarding the loss function for the line finder framework

In [18]:
import torch
import torch.nn as nn
from scipy.optimize import linear_sum_assignment

In [19]:
pred = torch.tensor([[56,124,0.4], [2,54,0.3], [102, 95, 0.99], [153,254, 0.3], [654,234, 0.2], [4,3450, 0.1], [345,6, 0.15], [23,634, 0.64], [345,64, 0.121]])
label = torch.tensor([[100,100], [200,200], [300,300]])

In [20]:
pred.shape

torch.Size([9, 3])

In [21]:
label.shape

torch.Size([3, 2])

In [22]:
crit = nn.MSELoss()

In [23]:
pred = pred.float()
label = label.float()

In [24]:
crit(label[0], pred[0,0:2])

tensor(1256.)

In [25]:
N = pred.shape[0]
M = label.shape[0]

In [26]:
cost = torch.zeros(N, M)

In [27]:
for n in range(N):
    for m in range(M):
        cost[n, m] = crit(pred[n,0:2], label[m])

In [28]:
X = linear_sum_assignment(cost)

In [29]:
cost

tensor([[1.2560e+03, 1.3256e+04, 4.5256e+04],
        [5.8600e+03, 3.0260e+04, 7.4660e+04],
        [1.4500e+01, 1.0314e+04, 4.0614e+04],
        [1.3262e+04, 2.5625e+03, 1.1862e+04],
        [1.6244e+05, 1.0364e+05, 6.4836e+04],
        [5.6159e+06, 5.3005e+06, 5.0051e+06],
        [3.4430e+04, 2.9330e+04, 4.4230e+04],
        [1.4554e+05, 1.0984e+05, 9.4142e+04],
        [3.0660e+04, 1.9760e+04, 2.8860e+04]])

In [13]:
X

(array([0, 2, 3], dtype=int64), array([0, 1, 2], dtype=int64))